# High Revenue, Low Profit Analysis (Very Strong Signal)

## Purpose
This notebook identifies products and sub-categories that generate **high revenue** but deliver **low or negative profit**. These represent critical business issues requiring immediate attention.

## Why This Matters
High-revenue, low-profit items are a **very strong signal** for business problems:
- They consume operational resources without delivering profitability
- They may indicate pricing issues, excessive discounting, or high cost of goods sold (COGS)
- Fixing these issues can have immediate impact on bottom-line profitability

## Analysis Steps
1. **Load Data**: Auto-detect CSV file in the data/ directory
2. **Map Columns**: Flexibly detect common column name variations
3. **Aggregate**: Sum sales and profit by Product and Sub-Category
4. **Flag Items**: Identify high-sales, low-profit combinations using configurable thresholds
5. **Visualize**: Create scatter plots highlighting flagged items
6. **Export**: Save flagged items to CSV for further investigation
7. **Recommend**: Provide actionable next steps

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configure matplotlib for inline plotting
%matplotlib inline

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

## Step 1: Load Data with Auto-Detection

We'll automatically detect CSV files in the data/ directory. This makes the notebook portable and easier to run.

In [ ]:
# Auto-detect CSV file in data/ directory
data_dir = Path('../data')
csv_files = list(data_dir.glob('*.csv'))

if not csv_files:
    raise FileNotFoundError(
        "No CSV files found in the data/ directory. "
        "Please ensure a CSV file exists at ../data/*.csv"
    )

# Use the first CSV file found
csv_path = csv_files[0]
print(f"Loading data from: {csv_path}")

# Load the data
df = pd.read_csv(csv_path)

print(f"\nData loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumn names:")
print(df.columns.tolist())

## Step 2: Column Mapping and Data Preparation

Different datasets may use different column names. We'll detect common variations and map them to standard names.

In [ ]:
# Define column name mappings for flexibility
# Maps common column name variations to standard names
column_mappings = {
    'product': ['Product', 'Product Name', 'ProductName', 'product_name', 'product'],
    'sub_category': ['Sub-Category', 'Sub Category', 'SubCategory', 'sub_category', 'subcategory'],
    'sales': ['Sales', 'sales', 'Revenue', 'revenue'],
    'profit': ['Profit', 'profit']
}

# Function to find the matching column in the dataframe
def find_column(df, possible_names):
    """Find the first matching column name from a list of possibilities."""
    for name in possible_names:
        if name in df.columns:
            return name
    return None

# Map columns
col_product = find_column(df, column_mappings['product'])
col_sub_category = find_column(df, column_mappings['sub_category'])
col_sales = find_column(df, column_mappings['sales'])
col_profit = find_column(df, column_mappings['profit'])

# Verify required columns exist
if not col_sales:
    raise ValueError("Could not find Sales/Revenue column in the dataset")
if not col_profit:
    raise ValueError("Could not find Profit column in the dataset")

print("Column mapping:")
print(f"  Product column: {col_product if col_product else 'NOT FOUND'}")
print(f"  Sub-Category column: {col_sub_category if col_sub_category else 'NOT FOUND'}")
print(f"  Sales column: {col_sales}")
print(f"  Profit column: {col_profit}")

# Create working dataframe with only needed columns
work_cols = [col_sales, col_profit]
if col_product:
    work_cols.insert(0, col_product)
if col_sub_category:
    work_cols.insert(1 if col_product else 0, col_sub_category)

df_work = df[work_cols].copy()

# Rename columns to standard names for easier processing
rename_dict = {col_sales: 'sales', col_profit: 'profit'}
if col_product:
    rename_dict[col_product] = 'product'
if col_sub_category:
    rename_dict[col_sub_category] = 'sub_category'

df_work.rename(columns=rename_dict, inplace=True)

print(f"\nWorking dataframe shape: {df_work.shape}")
print(f"\nFirst few rows:")
df_work.head()

In [ ]:
# Convert sales and profit to numeric, handling any non-numeric values
df_work['sales'] = pd.to_numeric(df_work['sales'], errors='coerce')
df_work['profit'] = pd.to_numeric(df_work['profit'], errors='coerce')

# Check for missing values before dropping
print("Missing values before cleaning:")
print(df_work.isnull().sum())

# Drop rows with missing sales or profit (these can't be analyzed)
rows_before = len(df_work)
df_work.dropna(subset=['sales', 'profit'], inplace=True)
rows_after = len(df_work)

print(f"\nRows dropped due to missing sales/profit: {rows_before - rows_after:,}")
print(f"Rows remaining: {rows_after:,}")

# Display basic statistics
print("\nBasic statistics:")
df_work[['sales', 'profit']].describe()

## Step 3: Aggregate by Product

We'll aggregate sales and profit at the product level to identify problematic products.

In [ ]:
# Aggregate by product if product column exists
if 'product' in df_work.columns:
    product_agg = df_work.groupby('product').agg({
        'sales': 'sum',
        'profit': 'sum',
        'product': 'size'  # Count number of orders
    }).rename(columns={'product': 'n_orders'})
    
    # Rename aggregated columns for clarity
    product_agg.columns = ['total_sales', 'total_profit', 'n_orders']
    
    # Sort by total sales descending
    product_agg = product_agg.sort_values('total_sales', ascending=False)
    
    print(f"Total unique products: {len(product_agg):,}")
    print(f"\nTop 10 products by sales:")
    print(product_agg.head(10))
    
    print(f"\nTop 10 products by profit:")
    print(product_agg.sort_values('total_profit', ascending=False).head(10))
    
    print(f"\nBottom 10 products by profit (most negative):")
    print(product_agg.sort_values('total_profit', ascending=True).head(10))
else:
    print("Product column not available - skipping product-level aggregation")
    product_agg = None

## Step 4: Aggregate by Sub-Category

Sub-category analysis provides a higher-level view of problem areas.

In [ ]:
# Aggregate by sub-category if column exists
if 'sub_category' in df_work.columns:
    subcategory_agg = df_work.groupby('sub_category').agg({
        'sales': 'sum',
        'profit': 'sum',
        'sub_category': 'size'  # Count number of orders
    }).rename(columns={'sub_category': 'n_orders'})
    
    # Rename aggregated columns for clarity
    subcategory_agg.columns = ['total_sales', 'total_profit', 'n_orders']
    
    # Sort by total sales descending
    subcategory_agg = subcategory_agg.sort_values('total_sales', ascending=False)
    
    print(f"Total unique sub-categories: {len(subcategory_agg):,}")
    print(f"\nAll sub-categories sorted by sales:")
    print(subcategory_agg)
else:
    print("Sub-Category column not available - skipping sub-category aggregation")
    subcategory_agg = None

## Step 5: Flagging Logic - Identify High Revenue, Low Profit Items

We'll use configurable thresholds to identify problematic items:
- **High Sales**: Top 20% by revenue (configurable)
- **Low Profit**: Bottom 20% by profit OR negative/zero profit (configurable)

Items that meet BOTH criteria are flagged as requiring immediate attention.

In [ ]:
# Configurable thresholds
HIGH_SALES_PERCENTILE = 80  # Top 20% = 80th percentile and above
LOW_PROFIT_PERCENTILE = 20  # Bottom 20% = 20th percentile and below

print(f"Configuration:")
print(f"  High sales threshold: Top {100-HIGH_SALES_PERCENTILE}% (>= {HIGH_SALES_PERCENTILE}th percentile)")
print(f"  Low profit threshold: Bottom {LOW_PROFIT_PERCENTILE}% (<= {LOW_PROFIT_PERCENTILE}th percentile) OR negative/zero")

def flag_items(agg_df, high_sales_pct=HIGH_SALES_PERCENTILE, low_profit_pct=LOW_PROFIT_PERCENTILE):
    """
    Flag items with high sales but low/negative profit.
    
    Parameters:
    - agg_df: Aggregated dataframe with total_sales and total_profit columns
    - high_sales_pct: Percentile threshold for high sales (default 80)
    - low_profit_pct: Percentile threshold for low profit (default 20)
    
    Returns:
    - DataFrame with flag columns added
    """
    df_flagged = agg_df.copy()
    
    # Calculate thresholds
    high_sales_threshold = df_flagged['total_sales'].quantile(high_sales_pct / 100)
    low_profit_threshold = df_flagged['total_profit'].quantile(low_profit_pct / 100)
    
    print(f"\nThreshold values:")
    print(f"  High sales threshold: ${high_sales_threshold:,.2f}")
    print(f"  Low profit threshold: ${low_profit_threshold:,.2f}")
    
    # Create flags
    df_flagged['high_sales_flag'] = df_flagged['total_sales'] >= high_sales_threshold
    df_flagged['low_or_negative_profit_flag'] = df_flagged['total_profit'] <= 0
    df_flagged['low_profit_bottom_pct_flag'] = df_flagged['total_profit'] <= low_profit_threshold
    
    # Strong signal: high sales AND (negative profit OR low profit)
    df_flagged['flagged_strong'] = (
        df_flagged['high_sales_flag'] & 
        (df_flagged['low_or_negative_profit_flag'] | df_flagged['low_profit_bottom_pct_flag'])
    )
    
    # Count flags
    print(f"\nFlag summary:")
    print(f"  High sales items: {df_flagged['high_sales_flag'].sum()}")
    print(f"  Low/negative profit items: {df_flagged['low_or_negative_profit_flag'].sum()}")
    print(f"  Low profit (bottom {low_profit_pct}%) items: {df_flagged['low_profit_bottom_pct_flag'].sum()}")
    print(f"  STRONG FLAG (high sales + low/negative profit): {df_flagged['flagged_strong'].sum()}")
    
    return df_flagged

# Apply flagging to product aggregation
if product_agg is not None:
    print("\n" + "="*60)
    print("PRODUCT LEVEL FLAGGING")
    print("="*60)
    product_flagged = flag_items(product_agg)
    
    print(f"\nStrongly flagged products:")
    strongly_flagged_products = product_flagged[product_flagged['flagged_strong']].sort_values('total_sales', ascending=False)
    print(strongly_flagged_products)
else:
    product_flagged = None
    strongly_flagged_products = None

In [ ]:
# Apply flagging to sub-category aggregation
if subcategory_agg is not None:
    print("\n" + "="*60)
    print("SUB-CATEGORY LEVEL FLAGGING")
    print("="*60)
    subcategory_flagged = flag_items(subcategory_agg)
    
    print(f"\nStrongly flagged sub-categories:")
    strongly_flagged_subcats = subcategory_flagged[subcategory_flagged['flagged_strong']].sort_values('total_sales', ascending=False)
    print(strongly_flagged_subcats)
else:
    subcategory_flagged = None
    strongly_flagged_subcats = None

## Step 6: Visualizations

Scatter plots help visualize the relationship between sales and profit, making it easy to spot problematic items.

In [ ]:
# Product-level scatter plot
if product_flagged is not None:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Plot non-flagged items in gray
    non_flagged = product_flagged[~product_flagged['flagged_strong']]
    ax.scatter(non_flagged['total_sales'], non_flagged['total_profit'], 
               alpha=0.4, s=50, color='gray', label='Normal')
    
    # Plot flagged items in red
    flagged = product_flagged[product_flagged['flagged_strong']]
    ax.scatter(flagged['total_sales'], flagged['total_profit'], 
               alpha=0.8, s=100, color='red', label='High Revenue, Low Profit', edgecolors='darkred', linewidths=1.5)
    
    # Add horizontal line at profit = 0
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    # Annotate top flagged items
    if len(flagged) > 0:
        top_n = min(5, len(flagged))  # Annotate up to 5 items
        for idx, row in flagged.head(top_n).iterrows():
            # Truncate long product names
            label = idx if len(str(idx)) < 30 else str(idx)[:27] + '...'
            ax.annotate(label, 
                       xy=(row['total_sales'], row['total_profit']),
                       xytext=(10, 10), textcoords='offset points',
                       fontsize=8, alpha=0.8,
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                       arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', alpha=0.6))
    
    ax.set_xlabel('Total Sales ($)', fontsize=12)
    ax.set_ylabel('Total Profit ($)', fontsize=12)
    ax.set_title('Product Analysis: Total Sales vs Total Profit\n(Red points = High Revenue, Low Profit)', 
                 fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Format axis labels with commas
    ax.ticklabel_format(style='plain', axis='both')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPlotted {len(product_flagged):,} products")
    print(f"Flagged: {len(flagged)} products with high sales but low/negative profit")
else:
    print("Product-level data not available - skipping product scatter plot")

In [ ]:
# Sub-category scatter plot
if subcategory_flagged is not None:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Plot non-flagged items in blue
    non_flagged = subcategory_flagged[~subcategory_flagged['flagged_strong']]
    ax.scatter(non_flagged['total_sales'], non_flagged['total_profit'], 
               alpha=0.6, s=150, color='steelblue', label='Normal')
    
    # Plot flagged items in red
    flagged = subcategory_flagged[subcategory_flagged['flagged_strong']]
    ax.scatter(flagged['total_sales'], flagged['total_profit'], 
               alpha=0.9, s=200, color='red', label='High Revenue, Low Profit', edgecolors='darkred', linewidths=2)
    
    # Add horizontal line at profit = 0
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    # Annotate all sub-categories (there aren't that many)
    for idx, row in subcategory_flagged.iterrows():
        color = 'red' if row['flagged_strong'] else 'black'
        weight = 'bold' if row['flagged_strong'] else 'normal'
        ax.annotate(idx, 
                   xy=(row['total_sales'], row['total_profit']),
                   xytext=(10, 5), textcoords='offset points',
                   fontsize=9, alpha=0.8, color=color, fontweight=weight,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7, edgecolor=color))
    
    ax.set_xlabel('Total Sales ($)', fontsize=12)
    ax.set_ylabel('Total Profit ($)', fontsize=12)
    ax.set_title('Sub-Category Analysis: Total Sales vs Total Profit\n(Red points = High Revenue, Low Profit)', 
                 fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPlotted {len(subcategory_flagged):,} sub-categories")
    print(f"Flagged: {len(flagged)} sub-categories with high sales but low/negative profit")
else:
    print("Sub-category data not available - skipping sub-category scatter plot")

In [ ]:
# Optional: Bar chart for flagged items (if any exist)
if product_flagged is not None and len(strongly_flagged_products) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Limit to top 15 for readability
    plot_data = strongly_flagged_products.head(15)
    
    # Sales bar chart
    plot_data['total_sales'].plot(kind='barh', ax=ax1, color='steelblue')
    ax1.set_xlabel('Total Sales ($)', fontsize=11)
    ax1.set_title('Top Flagged Products by Sales', fontsize=12, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    # Profit bar chart (note: these will be low/negative)
    colors = ['red' if x < 0 else 'orange' for x in plot_data['total_profit']]
    plot_data['total_profit'].plot(kind='barh', ax=ax2, color=colors)
    ax2.set_xlabel('Total Profit ($)', fontsize=11)
    ax2.set_title('Top Flagged Products by Profit (Low/Negative)', fontsize=12, fontweight='bold')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
elif subcategory_flagged is not None and len(strongly_flagged_subcats) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Sales bar chart
    strongly_flagged_subcats['total_sales'].plot(kind='barh', ax=ax1, color='steelblue')
    ax1.set_xlabel('Total Sales ($)', fontsize=11)
    ax1.set_title('Flagged Sub-Categories by Sales', fontsize=12, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    # Profit bar chart
    colors = ['red' if x < 0 else 'orange' for x in strongly_flagged_subcats['total_profit']]
    strongly_flagged_subcats['total_profit'].plot(kind='barh', ax=ax2, color=colors)
    ax2.set_xlabel('Total Profit ($)', fontsize=11)
    ax2.set_title('Flagged Sub-Categories by Profit (Low/Negative)', fontsize=12, fontweight='bold')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No flagged items to display in bar chart")

## Step 7: Export Results

Save flagged items to CSV files for further investigation and sharing with stakeholders.

In [ ]:
# Create output directory if it doesn't exist
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

print(f"Output directory: {output_dir.absolute()}")

# Export flagged products
if product_flagged is not None and len(strongly_flagged_products) > 0:
    output_path = output_dir / 'high_revenue_low_profit_products.csv'
    strongly_flagged_products.to_csv(output_path)
    print(f"\nExported {len(strongly_flagged_products)} flagged products to: {output_path}")
else:
    print("\nNo flagged products to export")

# Export flagged sub-categories
if subcategory_flagged is not None and len(strongly_flagged_subcats) > 0:
    output_path = output_dir / 'high_revenue_low_profit_subcategories.csv'
    strongly_flagged_subcats.to_csv(output_path)
    print(f"Exported {len(strongly_flagged_subcats)} flagged sub-categories to: {output_path}")
else:
    print("No flagged sub-categories to export")

print("\nExport complete!")

## Step 8: Interpretation and Actionable Recommendations

### What Do These Flags Mean?

**High Revenue, Low Profit** items are a critical business concern because:

1. **They consume resources without delivering value**: These products/categories generate significant sales volume but little or negative profit
2. **They indicate operational inefficiencies**: High sales with low profit suggests pricing, discounting, or cost issues
3. **They represent immediate improvement opportunities**: Fixing these issues can have quick, measurable impact

### Understanding the Flags

- **`high_sales_flag`**: Item is in the top 20% by total sales (configurable)
- **`low_or_negative_profit_flag`**: Item has zero or negative total profit
- **`low_profit_bottom_pct_flag`**: Item is in the bottom 20% by total profit (configurable)
- **`flagged_strong`**: Item meets BOTH high sales AND low/negative profit criteria

### Recommended Next Steps

#### 1. **Deep Dive into Flagged Items**
   - Review individual transactions for flagged products/sub-categories
   - Look for patterns: Are specific regions, customers, or time periods responsible?
   - Check for data quality issues (e.g., incorrect profit calculations)

#### 2. **Investigate Root Causes**
   
   **Discounting Issues:**
   - Calculate average discount % for flagged items
   - Compare discount rates between profitable and unprofitable products
   - Identify if excessive discounting is eroding margins
   
   **Cost Structure:**
   - Review Cost of Goods Sold (COGS) for flagged items
   - Check if supplier costs have increased
   - Analyze shipping costs, returns, or other operational expenses
   
   **Pricing:**
   - Compare pricing to market rates and competitors
   - Evaluate if prices are too low relative to costs
   - Consider if pricing strategy aligns with brand positioning

#### 3. **Time-Based Analysis**
   - Break down flagged items by month or quarter
   - Identify if profitability issues are getting worse or improving
   - Check for seasonal patterns that might explain low margins

#### 4. **Calculate and Monitor Profit Margins**
   - Compute profit margin % = (Profit / Sales) × 100
   - Set minimum acceptable margin thresholds by category
   - Create alerts for items that fall below thresholds

#### 5. **Take Corrective Actions**
   
   **Short-term (0-3 months):**
   - Reduce or eliminate discounts on flagged items
   - Implement minimum margin rules for sales teams
   - Negotiate better rates with suppliers
   - Consider temporary price increases
   
   **Medium-term (3-6 months):**
   - Restructure pricing strategy for flagged categories
   - Optimize product mix to favor profitable items
   - Improve operational efficiency to reduce costs
   - Explore alternative suppliers or manufacturing
   
   **Long-term (6+ months):**
   - Consider discontinuing consistently unprofitable products
   - Develop new, higher-margin product lines
   - Invest in customer education to justify premium pricing
   - Build brand value to support higher margins

#### 6. **Monitor Progress**
   - Re-run this analysis monthly or quarterly
   - Track changes in flag counts and profit margins
   - Measure the financial impact of corrective actions
   - Adjust thresholds and strategies based on results

### Key Metrics to Track

For each flagged item, monitor:
- **Profit Margin %**: Target minimum thresholds (e.g., 15-20% for retail)
- **Average Discount %**: Track over time, set maximum limits
- **Units Sold**: Understand volume dynamics
- **Return Rate**: High returns can erode profitability
- **Customer Concentration**: Are a few customers driving losses?

### Customization Options

You can adjust the flagging thresholds at the top of Step 5:
- `HIGH_SALES_PERCENTILE`: Currently 80 (top 20%). Increase for more selective flagging.
- `LOW_PROFIT_PERCENTILE`: Currently 20 (bottom 20%). Decrease for more selective flagging.

### Conclusion

This analysis provides a **very strong signal** about where profit improvement opportunities exist. The items flagged here should be prioritized for immediate investigation and corrective action. Even small improvements in margin on high-revenue items can have significant bottom-line impact.

**Remember:** High revenue is good, but **profitable revenue** is what drives business success!

## Summary Statistics

Let's calculate some final summary statistics for the analysis.

In [ ]:
print("="*70)
print("ANALYSIS SUMMARY")
print("="*70)

# Overall statistics
total_sales = df_work['sales'].sum()
total_profit = df_work['profit'].sum()
overall_margin = (total_profit / total_sales * 100) if total_sales > 0 else 0

print(f"\nOverall Business Metrics:")
print(f"  Total Sales: ${total_sales:,.2f}")
print(f"  Total Profit: ${total_profit:,.2f}")
print(f"  Overall Profit Margin: {overall_margin:.2f}%")
print(f"  Total Transactions: {len(df_work):,}")

# Product-level summary
if product_flagged is not None:
    flagged_sales = strongly_flagged_products['total_sales'].sum()
    flagged_profit = strongly_flagged_products['total_profit'].sum()
    
    print(f"\nProduct-Level Results:")
    print(f"  Total Products Analyzed: {len(product_flagged):,}")
    print(f"  Flagged Products: {len(strongly_flagged_products)}")
    print(f"  Flagged Products % of Total: {len(strongly_flagged_products)/len(product_flagged)*100:.1f}%")
    print(f"  Flagged Products Sales: ${flagged_sales:,.2f} ({flagged_sales/total_sales*100:.1f}% of total)")
    print(f"  Flagged Products Profit: ${flagged_profit:,.2f}")
    
    if flagged_sales > 0:
        potential_improvement = abs(flagged_profit)
        print(f"\n  💡 OPPORTUNITY: If flagged products achieved just 10% margin:")
        print(f"     Potential profit improvement: ${flagged_sales * 0.10 - flagged_profit:,.2f}")

# Sub-category summary
if subcategory_flagged is not None:
    flagged_sales_sc = strongly_flagged_subcats['total_sales'].sum()
    flagged_profit_sc = strongly_flagged_subcats['total_profit'].sum()
    
    print(f"\nSub-Category Level Results:")
    print(f"  Total Sub-Categories Analyzed: {len(subcategory_flagged)}")
    print(f"  Flagged Sub-Categories: {len(strongly_flagged_subcats)}")
    if len(subcategory_flagged) > 0:
        print(f"  Flagged Sub-Categories % of Total: {len(strongly_flagged_subcats)/len(subcategory_flagged)*100:.1f}%")
    print(f"  Flagged Sub-Categories Sales: ${flagged_sales_sc:,.2f} ({flagged_sales_sc/total_sales*100:.1f}% of total)")
    print(f"  Flagged Sub-Categories Profit: ${flagged_profit_sc:,.2f}")

print("\n" + "="*70)
print("Analysis complete! Review flagged items and take action.")
print("="*70)